In [7]:
import pandas as pd 
import numpy as np

#LOADING THE DATA
df=pd.read_csv(r"C:\Users\riya choudhary\OneDrive\Desktop\Python dataset\smart_city_data.csv")

#PREVIEW THE DATA
df.info()
print('\n',df.head())
print("\nShape of the data (rows, columns):",df.shape)
print("\nData types of columns:\n",df.dtypes)
print("\nDimension of the dataset:",df.ndim)

#SUMMARY STATISTICS
print("\nSummary Statistics:\n",df.describe())

#CHECKING FOR MISSING VALUES
print("\nMissing values:")
print(df.isnull().sum())

#CHECKING FOR DUPLICATES
print("\nDuplicate values:")
print(df.duplicated().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 14 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   City_State                           1500 non-null   object 
 1   Timestamp                            1500 non-null   object 
 2   Traffic_Congestion (%)               1500 non-null   int64  
 3   Avg_Vehicle_Speed (MPH)              1500 non-null   int64  
 4   PM2.5_Level (µg/m³)                  1500 non-null   float64
 5   CO2_Emissions (Metric Tons)          1500 non-null   float64
 6   Noise_Pollution (dB)                 1500 non-null   int64  
 7   Renewable_Energy (%)                 1500 non-null   float64
 8   Electricity_Consumption (MWh)        1500 non-null   float64
 9   Water_Consumption (Million Gallons)  1500 non-null   float64
 10  Temperature (°F)                     1500 non-null   float64
 11  Sensor_Reliability (0-1)      

In [6]:
#EXPLORING CATEGORICAL COLUMNS
# Unique values and frequency for each categorical column
print("\nTop Cities with Most Records:\n", df['City_State'].value_counts().head(10))
print("\nUnique Cities:", df['City_State'].nunique())

print("\nTraffic Incident Types:\n", df['Traffic_Incident'].value_counts())
print("\nUnique Traffic Incidents:", df['Traffic_Incident'].nunique())

print("\nWeather Conditions:\n", df['Weather_Condition'].value_counts())
print("\nUnique Weather Conditions:", df['Weather_Condition'].nunique())



Top Cities with Most Records:
 City_State
Houston, TX         174
Philadelphia, PA    168
Chicago, IL         161
Phoenix, AZ         153
San Jose, CA        152
Dallas, TX          151
New York, NY        151
San Antonio, TX     138
San Diego, CA       127
Los Angeles, CA     125
Name: count, dtype: int64

Unique Cities: 10

Traffic Incident Types:
 Traffic_Incident
Clear                333
Heavy Traffic        308
Event Nearby         300
Road Construction    294
Accident             265
Name: count, dtype: int64

Unique Traffic Incidents: 5

Weather Conditions:
 Weather_Condition
Snowy     323
Cloudy    304
Rainy     295
Sunny     291
Foggy     287
Name: count, dtype: int64

Unique Weather Conditions: 5
